In [1]:
#importing libraries
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
import numpy as np


driver = webdriver.Chrome()

#scrapping the name of stocks from bse website
url = "https://m.bseindia.com/sensex_new.aspx?Scripflag=22"
driver.get(url)
driver.implicitly_wait(10)
html = driver.page_source
soup = bs(html, 'html.parser')
all_company= soup.find_all(class_ ="TTRow_left")


df_final = pd.DataFrame(columns=['Name', 'Price', 'Change'])

for names in all_company:
    name = names.text

    # Scrape data for the current company from yahoo finance
   
    url = "https://finance.yahoo.com/quote/{}.NS/key-statistics?p={}.NS".format(name, name)
    driver.get(url)
    driver.implicitly_wait(30)
    html = driver.page_source
    soup = bs(html, 'html.parser')

        # Find the div tag with the key statistics
    key_stats_div = soup.find('div', {'id': 'Col1-0-KeyStatistics-Proxy'})

    #remove extra tags from column name
    for tags in key_stats_div.find_all("sup"):
        tags.decompose()
       
    # Get data from key statistics div
    key_stats_var = key_stats_div.find_all("td")
    data = {}
    for i in range(0, len(key_stats_var), 2):
        data[key_stats_var[i].text] = key_stats_var[i + 1].text

        # Find the name, price and change        
    stock  = soup.find(class_ ="D(ib) Fz(18px)").get_text()
    price  = soup.find(class_ ="Fw(b) Fz(36px) Mb(-4px) D(ib)").get_text()
    change = soup.find(class_ ="Fw(500) Pstart(8px) Fz(24px)").get_text()
        # Add the results to the DataFrame
    data['Name'] = stock
    data['Price'] = float(price.replace(',',''))
    data['Change'] = change
    df = pd.DataFrame(data, index=[0])
    df_final = pd.concat([df_final, df], ignore_index=True)
    #break
    

driver.quit()
df_final

#replace NA to NAN
df_final.replace('N/A', np.nan, inplace=True)


# convert all columns except for name column
for col in df_final.columns:
    df_final[col] = pd.to_numeric(df_final[col], errors ='ignore') 


#removing extra string from column name    
new_columns = df_final.columns.str.split('(').str.get(0).str.strip()
df_final.columns =  new_columns

    
df_final['Book Value Per Share'] = pd.to_numeric(df_final['Book Value Per Share'], errors='coerce')
df_final['Profit Margin'] = df_final['Profit Margin'].str.strip("%").astype(float)
#df_final['50-Day Moving Average'] = df_final['50-Day Moving Average'].str.replace(",","").astype(float)
df_final['50-Day Moving Average'] = df_final['50-Day Moving Average'].str.replace(",","").astype(float)


#creating screener 
screener = df_final[(df_final['Trailing P/E'] < 20)  &                         #undervalued
                    (df_final['Price/Book'] < 2) &                             #undervalued
                    (df_final['Profit Margin'] > 5 ) &                         # for profitability
                    (df_final['50-Day Moving Average'] > df_final['Price'])]   # for trend
screener

# saving data in Excel file
writer = pd.ExcelWriter('Top_100_Stocks.xlsx')

# write the first sheet
df_final.to_excel(writer, sheet_name='All_data', index=False)
#writing in 2nd sheet
screener.to_excel(writer, sheet_name='Screener', index=False)

# save the file
writer.save()

<ipython-input-1-8db59ae0ea10>:90: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  writer = pd.ExcelWriter('Top_100_Stocks.xlsx')
<ipython-input-1-8db59ae0ea10>:98: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
